# Annotation of GENIE mutations

In [1]:
import os
import pandas as pd
import nirvanaparser as nip
from tqdm.auto import tqdm
import gzip

## Input files

In [2]:
genie_dir = "../Data/Original/genie-15.0"
vcf_file = os.path.join(genie_dir, "genie_normalized.vcf.bgz")
json_file = "../Results/genie_normalized.json.gz"
cosmic_cmc_file = os.path.join("../Data/Original", "cmc_export.tsv.gz")

## Output files

In [3]:
vcf_id_to_vid_file = os.path.join(genie_dir, "genie.vcf_id_to_hgvsg.tsv.gz")
annot_file = os.path.join(genie_dir, "genie.annot.tsv.gz")

## Nirvana version and annotation data sources

In [4]:
nip.get_header_scalars(json_file)

,Attribute,GENIE
0,annotator,Nirvana 3.21.0
1,creationTime,2024-03-15 13:55:27
2,genomeAssembly,GRCh37
3,schemaVersion,6
4,dataVersion,73


In [5]:
nip.get_data_sources(json_file)

,name,version,description,releaseDate
0,Ensembl,110,Ensembl GRCh37 Release,2023-02-08
1,RefSeq,105.20220307,NCBI Homo sapiens Annotation Release,2022-03-10
2,ClinVar,20231028,"A freely accessible, public archive of reports...",2023-11-05
3,COSMIC,98,resource for exploring the impact of somatic m...,2023-05-23
4,dbSNP,156,Identifiers for observed variants,2023-09-28
5,dbSNP,151,Identifiers for observed variants,2018-04-23
6,GME,20160618,Greater Middle Eastern Variome allele frequenc...,2016-06-18
7,gnomAD,2.1,Allele frequencies from Genome Aggregation Dat...,2018-10-17
8,MITOMAP,20200819,Small variants in the MITOMAP human mitochondr...,2020-08-19
9,1000 Genomes Project,Phase 3 v5a,A public catalogue of human variation and geno...,2013-05-27


## Get a mapping from Niravana variant ids to VCF-IDs

### Function for reading a VCF file

In [6]:
def read_vcf_gz(file_name):
    with gzip.open(file_name, "rt") as f:
        header = f.readline()
        while not header.startswith("#CHROM"):
            header = f.readline()
        header = header[1:].strip().split("\t")
    df = pd.read_csv(file_name, comment="#", header=None, sep="\t", compression="gzip")
    df.columns = header
    return df

### Read the vcf file 

In [7]:
vcf = read_vcf_gz(vcf_file)

In [8]:
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,GENIE
0,chr1,1684345,c431afab69588f3150a378d08fc0bf8713584d5eb092b9...,A,T,.,PASS,.,GT:VF,1/1:1
1,chr1,1684347,adc1644ed30da5964fd03a592c73e5ddf0d78e924e62d4...,C,CCCTCCT,.,PASS,.,GT:VF,1/1:1
2,chr1,1684347,a86d0a0f81cff8daa9f32b83497a784463aab899b34f84...,CCCTCCT,C,.,PASS,.,GT:VF,1/1:1
3,chr1,1684391,97b2d690a7032356e8d65641b60d3fb891fd1770e1c1b6...,C,A,.,PASS,.,GT:VF,1/1:1
4,chr1,1684395,15ae11efa9729592af42a0f345ee339da9979e129d44a7...,C,T,.,PASS,.,GT:VF,1/1:1
...,...,...,...,...,...,...,...,...,...,...
862092,chrY,21894633,4d2d45c22fb442113473f832cecc3e47544ad8a91362c3...,CAA,C,.,PASS,.,GT:VF,1/1:1
862093,chrY,21894633,992feda83adb1109d0569dfb56aa523112d5abf4f80cf3...,CAAA,C,.,PASS,.,GT:VF,1/1:1
862094,chrY,21897244,dff14a4f48c0a693839b97c747dc329e97d32cc2fc537f...,G,T,.,PASS,.,GT:VF,1/1:1
862095,chrY,21901521,4529089ce4ace971769eaff8da439021a06c49d903692a...,C,G,.,PASS,.,GT:VF,1/1:1


### Get a mapping of CHROM, POS, REF, ALT to Nirvana variant ID

All positions from Nirvana's output are scanned and a mapping table from the original specification of chromosome, position, reference and alternative allele (from input VCF file) to Nirvana's variant identifier is created.

In [9]:
pos_all = nip.get_positions(json_file)

In [10]:
rows = []
for p in tqdm(pos_all):
    if len(p["altAlleles"]) != 1:
        raise ValueError("Not exactly one refAllele per position")
    if len(p["variants"]) != 1:
        raise ValueError("Not exactly one variant per position")
    rows.append(
        {
            "CHROM": p["chromosome"],
            "POS": p["position"],
            "REF": p["refAllele"],
            "ALT": p["altAlleles"][0],
            "vid": p["variants"][0]["vid"],
            "hgvsg": p["variants"][0].get("hgvsg", pd.NA),
            "sample": p["samples"][0]["sampleId"],
        }
    )
cpra_to_vid = pd.DataFrame(rows)
del rows

  0%|          | 0/862097 [00:00<?, ?it/s]

In [11]:
cpra_to_vid

,CHROM,POS,REF,ALT,vid,hgvsg,sample
0,chr1,1684345,A,T,1-1684345-A-T,NC_000001.10:g.1684345A>T,GENIE
1,chr1,1684347,C,CCCTCCT,1-1684347-C-CCCTCCT,NC_000001.10:g.1684370_1684375dup,GENIE
2,chr1,1684347,CCCTCCT,C,1-1684347-CCCTCCT-C,NC_000001.10:g.1684370_1684375del,GENIE
3,chr1,1684391,C,A,1-1684391-C-A,NC_000001.10:g.1684391C>A,GENIE
4,chr1,1684395,C,T,1-1684395-C-T,NC_000001.10:g.1684395C>T,GENIE
...,...,...,...,...,...,...,...
862092,chrY,21894633,CAA,C,Y-21894633-CAA-C,NC_000024.9:g.21894647_21894648del,GENIE
862093,chrY,21894633,CAAA,C,Y-21894633-CAAA-C,NC_000024.9:g.21894646_21894648del,GENIE
862094,chrY,21897244,G,T,Y-21897244-G-T,NC_000024.9:g.21897244G>T,GENIE
862095,chrY,21901521,C,G,Y-21901521-C-G,NC_000024.9:g.21901521C>G,GENIE


### Create a mapping table of VCF id to hgvsg specification

In [12]:
df = vcf.copy().drop(columns=["QUAL", "FILTER", "INFO", "FORMAT"])
df = df.melt(
    id_vars=["ID", "CHROM", "POS", "REF", "ALT"], var_name="sample", value_name="mut"
)
df["mut"] = df["mut"].str.startswith("1/1")
df = df.loc[df.mut].drop(columns="mut").reset_index(drop=True)
id_to_hgvsg = df.merge(
    cpra_to_vid, how="left", on=["CHROM", "POS", "REF", "ALT", "sample"]
).drop_duplicates()
id_to_hgvsg

,ID,CHROM,POS,REF,ALT,sample,vid,hgvsg
0,c431afab69588f3150a378d08fc0bf8713584d5eb092b9...,chr1,1684345,A,T,GENIE,1-1684345-A-T,NC_000001.10:g.1684345A>T
1,adc1644ed30da5964fd03a592c73e5ddf0d78e924e62d4...,chr1,1684347,C,CCCTCCT,GENIE,1-1684347-C-CCCTCCT,NC_000001.10:g.1684370_1684375dup
2,a86d0a0f81cff8daa9f32b83497a784463aab899b34f84...,chr1,1684347,CCCTCCT,C,GENIE,1-1684347-CCCTCCT-C,NC_000001.10:g.1684370_1684375del
3,97b2d690a7032356e8d65641b60d3fb891fd1770e1c1b6...,chr1,1684391,C,A,GENIE,1-1684391-C-A,NC_000001.10:g.1684391C>A
4,15ae11efa9729592af42a0f345ee339da9979e129d44a7...,chr1,1684395,C,T,GENIE,1-1684395-C-T,NC_000001.10:g.1684395C>T
...,...,...,...,...,...,...,...,...
864954,4d2d45c22fb442113473f832cecc3e47544ad8a91362c3...,chrY,21894633,CAA,C,GENIE,Y-21894633-CAA-C,NC_000024.9:g.21894647_21894648del
864955,992feda83adb1109d0569dfb56aa523112d5abf4f80cf3...,chrY,21894633,CAAA,C,GENIE,Y-21894633-CAAA-C,NC_000024.9:g.21894646_21894648del
864956,dff14a4f48c0a693839b97c747dc329e97d32cc2fc537f...,chrY,21897244,G,T,GENIE,Y-21897244-G-T,NC_000024.9:g.21897244G>T
864957,4529089ce4ace971769eaff8da439021a06c49d903692a...,chrY,21901521,C,G,GENIE,Y-21901521-C-G,NC_000024.9:g.21901521C>G


Assert that all variant IDs from the VCF file have a mapping to a HGVSG:

In [13]:
len(set(vcf.ID) - set(id_to_hgvsg.ID))

0

### Write the mapping table of VCF id to hgvsg

In [14]:
id_to_hgvsg.to_csv(vcf_id_to_vid_file, sep="\t", index=False)

## Read the Cosmic Cancer Mutation Census file

In [15]:
def tier_rank(x):
    tier_ranks = {"1": 1, "2": 2, "3": 3, "Other": 4, pd.NA: 5}
    return tier_ranks[x]

In [16]:
cmc = pd.read_table(cosmic_cmc_file, encoding="latin1", low_memory=False)
cmc = cmc[cmc.GENOMIC_MUTATION_ID.notna()]
cmc = cmc[
    ["GENOMIC_MUTATION_ID", "GENE_NAME", "Mutation CDS", "ONC_TSG", "CGC_TIER", "MUTATION_SIGNIFICANCE_TIER"]
].drop_duplicates()
cmc = cmc.groupby(["GENOMIC_MUTATION_ID", "GENE_NAME", "Mutation CDS", "ONC_TSG", "CGC_TIER"])[
    "MUTATION_SIGNIFICANCE_TIER"
].apply(lambda x: min(x, key=tier_rank))
cmc = cmc.reset_index().set_index(["GENOMIC_MUTATION_ID", "Mutation CDS"], drop=True).sort_index()
cmc["CGC_TIER"] = cmc["CGC_TIER"].astype("int").astype("str")
cmc

,,GENE_NAME,ONC_TSG,CGC_TIER,MUTATION_SIGNIFICANCE_TIER
GENOMIC_MUTATION_ID,Mutation CDS,,,,
COSV100000072,c.206T>C,MYOD1,oncogene,1,Other
COSV100000124,c.177C>A,MYOD1,oncogene,1,Other
COSV100000125,c.355C>T,MYOD1,oncogene,1,Other
COSV100000126,c.296A>G,MYOD1,oncogene,1,Other
COSV100000133,c.790C>A,MYOD1,oncogene,1,Other
...,...,...,...,...,...
COSV99994110,c.122C>A,CEP89,fusion,2,Other
COSV99994134,c.64C>T,CEP89,fusion,2,Other
COSV99994197,c.889G>A,CEP89,fusion,2,Other


## Get annotated table of individual mutations

In [17]:
pos = nip.filter_positions_by_variants(pos_all, lambda v: "transcripts" in v)
# pos = nip.filter_positions_by_variants(pos, lambda v: nip.common_variant_filter(v, 5e-4))

In [18]:
len(pos_all)

862097

In [19]:
len(pos)

861517

In [20]:
mt = nip.get_mutation_table_for_positions(pos)

Positions:   0%|          | 0/861517 [00:00<?, ?it/s]

### Add Cosmic Cancer Gene and Mutation Census Tiers

In [21]:
mutation_tier_ranks = {"1": 1, "2": 2, "3": 3, "Other": 4, pd.NA: 5}
gene_tier_ranks = {"1": 1, "2": 2, pd.NA: 3}
cmc_tiers = []
for p in tqdm(pos):
    for v in p["variants"]:
        vid = v["vid"]
        for t in v.get("transcripts", []):
            best_mutation_tier = pd.NA
            best_gene_type = pd.NA
            best_gene_tier = pd.NA
            best_gene_name = pd.NA
            # Only investigate transcripts with a cDNA change
            if "hgvsc" not in t:
                continue
            hgvsc_short = t["hgvsc"].split(":")[1]
            for cosmic_entry in v.get("cosmic", []):
                # Only keep COSMIC entries for the same genomic change
                if not cosmic_entry.get("isAlleleSpecific", False):
                    continue
                # Only keep COSMIC entries which were found in the Cancer Mutation Census
                cosmic_id = cosmic_entry["id"]
                if cosmic_id not in cmc.index:
                    continue
                cmc_entries = cmc.loc[cosmic_id]
                #if hgvsc_short not in cmc_entries.index:
                #    continue
                #cmc_entries = cmc_entries.loc[[hgvsc_short]]
                for idx, cmc_entry in cmc_entries.iterrows():
                    cmc_tier = cmc_entry.MUTATION_SIGNIFICANCE_TIER
                    cgc_gene_type = cmc_entry.ONC_TSG
                    cgc_tier = cmc_entry.CGC_TIER
                    cgc_gene = cmc_entry.GENE_NAME
                    if mutation_tier_ranks[cmc_tier] < mutation_tier_ranks[best_mutation_tier]:
                        best_mutation_tier = cmc_tier
                    if gene_tier_ranks[cgc_tier] < gene_tier_ranks[best_gene_tier]:
                        best_gene_type = cgc_gene_type
                        best_gene_tier = cgc_tier
                        best_gene_name = cgc_gene
            cmc_tiers.append({
                "vid": vid,
                "hgvsc": t["hgvsc"],
                "cosmicGeneName": best_gene_name,
                "cosmicGeneType": best_gene_type,
                "cosmicGeneTier": best_gene_tier,
                "cosmicMutationTier": best_mutation_tier
            })
cmc_tiers = pd.DataFrame(cmc_tiers)

  0%|          | 0/861517 [00:00<?, ?it/s]

In [22]:
mt = mt.merge(cmc_tiers, on=["vid", "hgvsc"], how="left")

In [23]:
mt.loc[
    (mt.hgvsp.isna() | (mt.hgvsp == "")) & ~((mt.hgnc == mt.cosmicGeneName) & (mt.cosmicMutationTier.notna())),
    ["cosmicGeneName", "cosmicGeneType", "cosmicGeneTier", "cosmicMutationTier"],
] = pd.NA

So far a cancer gene cencus tier is only added if a variant has an assigned Cosmic entry. As a consequence, some cancer genes do not get a tier assigned if the variants found in the samples are not included in the cancer mutation census. Here we add an the tier based on gene names for those variants that don't have a tier assigned yet.

In [24]:
cmc_gene_to_tier = (
    cmc[["GENE_NAME", "ONC_TSG", "CGC_TIER"]].drop_duplicates().set_index("GENE_NAME", drop=True)
)
mt = mt.merge(cmc_gene_to_tier, how="left", left_on="hgnc", right_index=True)
subset = mt.cosmicGeneTier.isna()
mt.loc[subset, "cosmicGeneType"] = mt.loc[subset, "ONC_TSG"]
mt.loc[subset, "cosmicGeneTier"] = mt.loc[subset, "CGC_TIER"]
mt = mt.drop(columns=["ONC_TSG", "CGC_TIER"])

### Add VEP consequence priority

In [25]:
def get_highest_vep_priority(consequences):
    if type(consequences) is str:
        consequences = consequences.split(",")
    hp = min([nip.get_vep_priority_for_consequence(c) for c in consequences])
    return hp

In [26]:
mt["consequence_priority"] = mt.consequence.map(
    lambda x: get_highest_vep_priority(x) if type(x) is list else x
)

### Write mutation table

In [36]:
mt["consequence"] = mt["consequence"].astype("str")
mt = mt.drop_duplicates()
mt.to_csv(annot_file, sep="\t", index=False)

## Timestamp

In [37]:
from datetime import datetime

datetime.now().astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")

'2024-03-21 18:57:21 CET'

In [38]:
from sinfo import sinfo
sinfo()

-----
nirvanaparser       0.2.129
pandas              2.1.4
sinfo               0.3.1
tqdm                4.66.1
-----
IPython             8.18.1
jupyter_client      8.6.0
jupyter_core        5.7.1
jupyterlab          4.0.10
notebook            7.0.6
-----
Python 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:43:22) [GCC 12.3.0]
Linux-5.13.0-1029-aws-x86_64-with-glibc2.31
8 logical CPU cores, x86_64
-----
Session information updated at 2024-03-21 18:57


In [39]:
mt.columns

Index(['sample', 'chromosome', 'position', 'genotype', 'variantFrequency',
       'hgnc', 'source', 'geneId', 'transcriptId', 'proteinId', 'hgvsc',
       'hgvsp', 'isCanonical', 'vid', 'hgvsg', 'begin', 'end', 'refAllele',
       'altAllele', 'variantType', 'bioType', 'geneType', 'mutationStatus',
       'codons', 'aminoAcids', 'cdnaPos', 'cdsPos', 'exons', 'proteinPos',
       'consequence', 'cosmicSampleCount', 'maxGnomadAf', 'maxGnomadExomeAf',
       'maxOneKG', 'cosmicGeneName', 'cosmicGeneType', 'cosmicGeneTier',
       'cosmicMutationTier', 'consequence_priority'],
      dtype='object')